In [1]:
import sys, os

sys.path.insert(0, "")
sys.path.append("../")
from utils.Utils import *

Welcome to JupyROOT 6.26/04


In [2]:
import uproot
import awkward as ak
from collections import OrderedDict
import numpy as np

from coffea import nanoevents
from coffea.nanoevents.methods.base import NanoEventsArray
from coffea.analysis_tools import Weights, PackedSelection
from coffea.nanoevents.methods import nanoaod
from coffea.nanoevents.methods import vector
from coffea.lookup_tools.dense_lookup import dense_lookup

In [3]:
import HHbbVV
from HHbbVV.processors.utils import P4
from HHbbVV import run_utils

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
run_utils.add_mixins(nanoevents)

events = nanoevents.NanoEventsFactory.from_root(
    "root://cmseos.fnal.gov///store/user/lpcpfnano/cmantill/v2_3/2018/HH/GluGluToHHTobbVV_node_cHHH1_TuneCP5_13TeV-powheg-pythia8/GluGluToHHTobbVV_node_cHHH1/220808_145933/0000/nano_mc2018_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()[:5000]

/home/users/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1937 branches) at 0x7f8a08af98e0>, taking first instance
  warnings.warn(
/home/users/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1937 branches) at 0x7f8a08af98e0>, taking first instance
  warnings.warn(
/home/users/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1937 branches) at 0x7f8a08af98e0>, taking first instance
  warnings.warn(
/home/users/rkansal/mambaforge/envs/python39/lib/python3.9/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1937 b

In [6]:
events = events[ak.count(events.FatJet.pt, axis=1) >= 2]

In [7]:
gen_weights = np.sign(events["genWeight"])
n_events = np.sum(gen_weights)

selection = PackedSelection()
cutflow = OrderedDict()
cutflow["all"] = n_events

selection_args = (selection, cutflow, False, gen_weights)

In [8]:
gen_vars_dict, (genbb, genq) = HHbbVV.processors.GenSelection.gen_selection_HHbbVV(
    events, events.FatJet, selection, cutflow, gen_weights, P4
)

In [9]:
vvfj1 = gen_vars_dict["ak8FatJetHVV"][:, 0].astype(bool)
vvfj4 = gen_vars_dict["ak8FatJetHVVNumProngs"] == 4
selector = vvfj1 * vvfj4

ak8_pfcands = events[selector].FatJetPFCands
ak8_pfcands = ak8_pfcands[ak8_pfcands.jetIdx == 0]
pfcands = events[selector].PFCands[ak8_pfcands.pFCandsIdx]
pfc4 = ak.fill_none((ak.count(pfcands.pt, axis=1) >= 4), False)
selector[selector] = pfc4
selector = np.ma.filled(selector, fill_value=False)

vv_fatjets = events.FatJet[selector][:, 0:1]
gen_quarks = genq[selector]

In [45]:
lpsfargs = [
    "2018",
    events[selector],
    vv_fatjets,
    0,
    4,
    gen_quarks,
    gen_weights[selector],
    "GluGluToHHTobbVV_node_cHHH1",
]

# lpsfs, lp_hist = HHbbVV.processors.corrections.get_lund_SFs(
# lds, kt_subjets_pt = HHbbVV.processors.corrections.get_lund_SFs(*lpsfargs)

## Oz version

In [11]:
f_ratio_name = "data/ratio_2018.root"
f_ratio = ROOT.TFile.Open(f_ratio_name)
LP_rw = LundReweighter(f_ratio=f_ratio)

In [12]:
ak8_pfcands = events[selector].FatJetPFCands
ak8_pfcands = ak8_pfcands[ak8_pfcands.jetIdx == 0]
pfcands = events[selector].PFCands[ak8_pfcands.pFCandsIdx]

pfcands_vector_ptetaphi = ak.Array(
    [
        [{kin_key: cand[kin_key] for kin_key in P4} for cand in event_cands]
        for event_cands in pfcands
    ],
    with_name="PtEtaPhiMLorentzVector",
)

In [13]:
gen_parts_eta_phi = [np.array([[gq.eta, gq.phi] for gq in genqs]) for genqs in gen_quarks]
pf_cands = [
    np.array([[pfc.px, pfc.py, pfc.pz, pfc.E] for pfc in pcands])
    for pcands in pfcands_vector_ptetaphi
]
ak8_jets = np.array([[fj.pt, fj.eta, fj.phi, fj.mass] for fj in vv_fatjets])

In [14]:
ozlpsfs = LP_rw.get_all_weights(pf_cands, gen_parts_eta_phi, ak8_jets)

In [41]:
reclust = LP_rw.get_all_weights(pf_cands, gen_parts_eta_phi, ak8_jets)

In [48]:
reclust.subjet.pt()

138.28075156217162

In [51]:
tpfc = np.array([[j.pt(), j.eta(), j.phi(), j.m()] for j in reclust.split])

In [52]:
len(tpfc)

18

In [60]:
print("pt")
print(np.sort(tpfc[:, 0]))
print("eta")
print(tpfc[:, 1][np.argsort(tpfc[:, 0])])

pt
[ 1.08105469  1.16699219  1.87890625  1.90136719  2.49609375  2.86328125
  2.89453125  3.70703125  3.828125    4.11328125  5.17578125  5.28515625
  8.53125     9.015625   13.0546875  15.4765625  18.         33.03125   ]
eta
[2.11474609 2.00732422 1.88720703 1.95385742 1.81274414 2.01464844
 1.73510742 1.82543945 1.78662109 1.8203125  1.80273438 1.81591797
 1.78393555 1.87451172 1.89013672 1.8527832  1.96264648 1.82177734]


In [61]:
_ = HHbbVV.processors.corrections.get_lund_SFs(*lpsfargs)

18
[ 1.08105469  1.16699219  1.87890625  1.90136719  2.49609375  2.86328125
  2.89453125  3.70703125  3.828125    4.11328125  5.17578125  5.28515625
  8.53125     9.015625   13.0546875  15.4765625  18.         33.03125   ]
[2.11474609 2.00732422 1.88720703 1.95385742 1.81274414 2.01464844
 1.73510742 1.82543945 1.78662109 1.8203125  1.80273438 1.81591797
 1.78393555 1.87451172 1.89013672 1.8527832  1.96264648 1.82177734]


In [22]:
# [i, pseudojet.subjet_pt, pseudojet.order, delta, kt]
reclust.split

[[0, 138.28075156217162, 1, 0.1956627601258767, 4.9420269538016655],
 [0, 138.28075156217162, 1, 0.1514388488073542, 3.8624906534529724],
 [0, 138.28075156217162, 1, 0.09953083770982661, 2.650131775631638],
 [0, 138.28075156217162, 1, 0.046830191907542486, 1.3235363753041678],
 [1, 117.50006102571415, 1, 0.16621638335369476, 8.607996134456698],
 [1, 117.50006102571415, 1, 0.15082923347344215, 1.8997184888330756],
 [1, 117.50006102571415, 1, 0.05398228744291181, 0.696076292379109],
 [1, 117.50006102571415, 1, 0.026755493322500153, 0.22723847822664928],
 [1, 117.50006102571415, 1, 0.0070739358315290626, 0.05349663972593853],
 [2, 78.42901791957688, 1, 0.04653936078386569, 1.2409341700760845],
 [2, 78.42901791957688, 1, 0.025145833862259527, 0.4844502055025937],
 [2, 78.42901791957688, 1, 0.03333531917529359, 0.2453663141304665],
 [2, 78.42901791957688, 1, 0.012506080728565343, 0.1062039824371135],
 [2, 78.42901791957688, 1, 0.022630861637597744, 0.07262208138783809],
 [3, 76.755227071258

In [29]:
kt_subjets_pt[0]

<Array [78.4, 76.8, 118, 138] type='4 * float64'>

In [38]:
[np.array(lds[2][0].Delta), lds[2][0].kt.to_numpy()]

[array([0.41254202, 0.22822895, 0.17873902, 0.14232485, 0.08770532,
        0.0542449 , 0.02673373, 0.00707394]),
 array([2.06754456, 2.39295678, 2.2512468 , 0.80304252, 2.33905017,
        0.69946257, 0.21011038, 0.05349664])]

In [31]:
lds[0].Delta[0]

<Array [0.151, 0.085, ... 0.0178, 0.0088] type='7 * float64'>

## Comparing

In [91]:
pfcands_vector_ptetaphi[0][1:5].pt

<Array [1.38, 3.02, 2.16, 1.02] type='4 * float64'>

In [90]:
LP_rw.get_splittings(pf_cands[0][:5], num_excjets=1)

1.3818359375
2.162109375
3.0234375
1.0185546875


([[8.143427015946497,
   1.5774425984579568,
   2.8163710340815755,
   1.6809782356400331]],
 [[0, 8.143427015946497, 1, 0.24898153446513818, 0.9932828876177898],
  [0, 8.143427015946497, 1, 0.13524651947033228, 0.1868885010258986],
  [0, 8.143427015946497, 1, 0.11627923691818055, 0.07369650855458905]])

In [78]:
import fastjet

In [97]:
jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 1000.0)
cluster = fastjet._pyjet.AwkwardClusterSequence(pfcands_vector_ptetaphi[0][1:5], jetdef)
lds = cluster.exclusive_jets_lund_declusterings(1)

In [100]:
jet_def = fj.JetDefinition(fj.kt_algorithm, 0.8)
cs = fj.ClusterSequence(pfcands_vector_ptetaphi[0][:5], jet_def)
ldskt = cs.exclusive_jets_lund_declusterings(1)

In [98]:
lds[0].Delta

<Array [0.433, 0.174, 0.138] type='3 * float64'>

In [101]:
ldskt[0].Delta

<Array [0.249, 0.135, 0.116] type='3 * float64'>

In [77]:
(
    pfcands_vector_ptetaphi[0][:5][0]
    + pfcands_vector_ptetaphi[0][:5][1]
    + pfcands_vector_ptetaphi[0][:5][2]
    + pfcands_vector_ptetaphi[0][:5][3]
    + pfcands_vector_ptetaphi[0][:5][4]
).pt

8.143427015946495

In [70]:
ak.sum(pfcands_vector_ptetaphi[0][:5])

30.5704345703125

In [16]:
lpsfs["lp_sf_lnN"][:, 0]

array([1.59664762, 1.72517502, 1.71007049, 2.55584574, 2.03695035,
       1.58330703, 0.5383184 , 0.81909859, 0.84185511, 2.19395876,
       1.22305071, 2.82662725, 0.70131367, 2.53019452, 0.79218096,
       1.58675444, 0.4281795 , 3.03538942, 1.22881413, 1.35652339,
       0.79204077, 0.32769871, 0.33464086, 0.46744648, 1.04027224,
       0.61499828, 0.65758526, 0.83731133, 1.10940933, 0.52479166,
       1.17785251, 0.5491181 , 0.74898654, 1.96120262, 0.50407565,
       1.98561716, 1.07194805, 1.01364815, 2.36186337, 0.74234056,
       0.76702797, 0.96581626, 0.84862125, 1.15393651, 0.25921842,
       0.52934843, 7.43776178, 0.40402088, 2.57607627, 2.1975534 ,
       0.27223372, 0.67855448, 1.39450169, 1.07934749, 1.28474534,
       1.17779517, 0.39536989, 1.05008006, 3.01214099, 1.06926167,
       0.70574486, 1.12219334, 1.09067678, 1.62399828, 0.67055219,
       2.77288294, 1.58473039, 2.0658462 , 0.47359353, 0.88764173,
       1.52273428, 0.28774986, 0.5345003 , 0.6078521 , 0.48580

In [18]:
ozlpsfs["nom"]

array([1.62632169, 1.82717808, 0.91546557, 1.93837607, 0.83245758,
       0.62114351, 0.41607224, 0.97186377, 2.83437432, 0.15637381,
       1.56576783, 0.79018058, 0.96665743, 0.19860135, 0.7336155 ,
       2.34300388, 0.70285453, 1.30197862, 0.52084472, 1.3455701 ,
       0.84260438, 0.48554992, 0.64262326, 1.48769367, 0.42802185,
       0.3711837 , 0.40266381, 1.95792066, 2.5541874 , 0.88939937,
       1.04214817, 1.00643185, 0.57284831, 1.67607046, 0.63892256,
       1.50622008, 0.97617031, 0.87225076, 2.15839329, 0.51078798,
       0.74474527, 0.56970972, 0.66962036, 1.08035623, 0.3852276 ,
       1.32316282, 2.47580554, 0.46748724, 0.92342514, 0.93346814,
       1.48472446, 0.44724066, 0.69618738, 0.60351258, 1.38610863,
       0.47605923, 0.35093332, 0.65515306, 1.38769367, 0.9708136 ,
       0.81401945, 0.65864305, 1.37329229, 1.44569856, 2.14340696,
       2.68104449, 0.99423492, 0.60244177, 0.31181099, 0.9249874 ,
       0.60483397, 0.49026283, 0.28604989, 1.53383501, 1.64591